In [108]:
import numpy as np
import scipy
import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits

In [148]:
np.random.seed(42)

X, y = load_digits(return_X_y=True)

scaled_X = (np.asfarray(X) / 255 * 0.99) + 0.01

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.3, random_state=42)

In [149]:
class NeuralNetwork:


    def __init__(self, input_nodes, hid_layer1, hid_layer2, hid_layer3, output_nodes, learning_rate):

        self.inodes = input_nodes
        self.hnodes1 = hid_layer1
        self.hnodes2 = hid_layer2
        self.hnodes3 = hid_layer3
        self.onodes = output_nodes

        self.w_input_1 = np.random.normal(0, pow(self.hnodes1, -0.5), (self.hnodes1, self.inodes))
        self.w_1_2 = np.random.normal(0, pow(self.hnodes2, -0.5), (self.hnodes2, self.hnodes1))
        self.w_2_3 = np.random.normal(0, pow(self.hnodes3, -0.5), (self.hnodes3, self.hnodes2))
        self.w_3_output = np.random.normal(0, pow(self.onodes, -0.5), (self.onodes, self.hnodes3))

        self.lr = learning_rate

        self.activation_function = lambda x: scipy.special.expit(x)

        pass

# =====================================================================================================

    def train(self, inputs_list, targets_list):

        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        inputs_input_1 = np.dot(self.w_input_1, inputs)
        outputs_input_1 = self.activation_function(inputs_input_1)

        inputs_1_2 = np.dot(self.w_1_2, outputs_input_1)
        outputs_1_2 = self.activation_function(inputs_1_2)

        inputs_2_3 = np.dot(self.w_2_3, outputs_1_2)
        outputs_2_3 = self.activation_function(inputs_2_3)

        inputs_3_output = np.dot(self.w_3_output, outputs_2_3)
        outputs_3_output = self.activation_function(inputs_3_output)

        output_errors = targets - outputs_3_output
        errors_2_3 = np.dot(self.w_3_output.T, output_errors)
        errors_1_2 = np.dot(self.w_2_3.T, errors_2_3)
        errors_input_1 = np.dot(self.w_1_2.T, errors_1_2)

        self.w_3_output += self.lr * np.dot((output_errors * outputs_3_output * (1 - outputs_3_output)), np.transpose(outputs_2_3))
        self.w_2_3 += self.lr * np.dot((errors_2_3 * outputs_2_3 * (1 - outputs_2_3)), np.transpose(outputs_1_2))
        self.w_1_2 += self.lr * np.dot((errors_1_2 * outputs_1_2 * (1 - outputs_1_2)), np.transpose(outputs_input_1))
        self.w_input_1 += self.lr * np.dot((errors_input_1 * outputs_input_1 * (1 - outputs_input_1)), np.transpose(inputs))

        pass

# =====================================================================================================

    def query(self, inputs_list):

        inputs = np.array(inputs_list, ndmin=2).T

        inputs_input_1 = np.dot(self.w_input_1, inputs)
        outputs_input_1 = self.activation_function(inputs_input_1)

        inputs_1_2 = np.dot(self.w_1_2, outputs_input_1)
        outputs_1_2 = self.activation_function(inputs_1_2)

        inputs_2_3 = np.dot(self.w_2_3, outputs_1_2)
        outputs_2_3 = self.activation_function(inputs_2_3)

        inputs_3_output = np.dot(self.w_3_output, outputs_2_3)
        outputs_3_output = self.activation_function(inputs_3_output)

        return outputs_3_output

In [150]:
epochs = 200
input_nodes = 64
hid_layer1 = 42
hid_layer2 = 28
hid_layer3 = 20
output_nodes = 10
learning_rate = 0.1

nn = NeuralNetwork(
    input_nodes = input_nodes,
    hid_layer1 = hid_layer1,
    hid_layer2 = hid_layer2,
    hid_layer3 = hid_layer3,
    output_nodes = output_nodes,
    learning_rate = learning_rate
)

In [151]:
for epoch in tqdm.tqdm(range(epochs)):
    for i in range(X_train.shape[0]):

        targets = np.zeros(output_nodes) + 0.01
        targets[int(y_train[i])] = 0.99

        nn.train(X_train[i], targets)

100%|██████████| 200/200 [00:21<00:00,  9.24it/s]


In [152]:
y_pred_test = [np.argmax(nn.query(i)) for i in X_test]
y_pred_train = [np.argmax(nn.query(i)) for i in X_train]

print(f"Train score: {accuracy_score(y_train, y_pred_train)}")
print(f"Test score: {accuracy_score(y_test, y_pred_test)}")

Train score: 0.9992044550517104
Test score: 0.9574074074074074
